This notebook considers:
1. __data.x2__ in data.py. It contains 69 features (initiator exclusive). 
2. From the result, we can find there is no difference with manual selected 23 features. We don't generate features in this notebook but we get rules for 23 features.
3. In this notebook, we compare the prediction order (sphere, worm, vesicle, other) with (vesicle, worm, sphere, other). There is no difference between select variables.

In [1]:
import data1 as data
import random
from common import *
from rules import *
from realkd.patch import RuleFit
from sklearn.model_selection import cross_validate, KFold
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import LeaveOneOut
import numpy as np
import matplotlib.colors as mcolors

In [2]:
import warnings
warnings.filterwarnings("ignore")

## Full phase prediction


In [3]:
from sklearn.linear_model import LogisticRegressionCV
from sklearn.multioutput import ClassifierChain
from multilabel import BinaryRelevanceClassifier, ProbabilisticClassifierChain
from gam import LogisticGAM

STATE = np.random.RandomState(seed=1000)

lr = LogisticRegressionCV(penalty='l1', solver='saga', random_state=STATE)
lr_ind = BinaryRelevanceClassifier(lr)
lr_chain = ClassifierChain(lr, order=[0, 2, 1, 3])
lr_pcc = ProbabilisticClassifierChain(lr) 

# gams not fixed, remove this part.
# gam_ind = BinaryRelevanceClassifier(LogisticGAM(lam=20.0, max_iter=250))
# gam_chain = ClassifierChain(LogisticGAM(lam=20.0, max_iter=250))
# gam_pcc = ProbabilisticClassifierChain(LogisticGAM(lam=20.0, max_iter=250)) 

rf = RandomForestClassifier(random_state=STATE, min_samples_leaf=1, n_estimators=100)
rf_ind = BinaryRelevanceClassifier(rf)
rf_chain = ClassifierChain(rf, order=[0, 2, 1, 3])
rf_pcc = ProbabilisticClassifierChain(rf)

# Rulefit
rufit_pcc = RuleFitWrapper(mode='chain')

full_estimators = [lr_ind, lr_pcc, lr_chain, rf_ind, rf_pcc, rf_chain, rufit_pcc]
full_names = ['LR_ind', 'LR_pcc', 'LR_chain', 'RanF_ind', 'RanF_pcc', 'Ranf_chain', 'Rufit_pcc']

This following code under "2.6 GHz 6-Core Intel Core i7" runs ~5 hours. You can simply use saved result to re-run the result. (See below instructions)

```
import pickle
cur_save=open('./' + 'interpolation_30folder' + '.p', 'rb')
interpolation = pickle.load(cur_save)
```
After running these three-line code, you can ignore the following __interpolation__ code and re-run the rest.

Due to the individual prediction accuracy

In [6]:
from common import Experiment, LogLikelihoodEvaluator
from sklearn.model_selection import KFold

print("Current Prediction Order is:", data.y.columns.tolist())
print('Num of predictors:, ', data.x1.shape[1])
interpolation = Experiment(full_estimators, 
                    full_names,
                    KFold(30, shuffle=True, random_state=STATE),
                    data.x1, data.y.replace(-1.0, 0.0),
                    groups=data.comp_ids.array, 
                    evaluators=['accuracy', LogLikelihoodEvaluator(2, neg=True)],
                    verbose=True).run()

Current Prediction Order is: ['sphere', 'worm', 'vesicle', 'other']
Num of predictors:,  23
Running experiment with 30 repetitions
******************************


In [11]:
# import pickle
# with open('interpolation_select_swvo.pkl', 'wb') as f:   
#     pickle.dump(interpolation, f)

In [12]:
df1 = interpolation.summary()
df1 = df1.rename(index={'RF_ind': 'RanF_ind', 'RF_pcc': 'RanF_pcc', 'Rf_chain': 'Ranf_chain'})
df1.index = [each +'_sel' for each in df1.index.tolist()]
df1 # selected predictors 69 variables

,mean_train_accuracy,std_train_accuracy,mean_test_accuracy,std_test_accuracy,mean_train_log loss,std_train_log loss,mean_test_log loss,std_test_log loss
LR_ind_sel,0.382866,0.011192,0.379123,0.129272,2.908696,0.010626,2.926368,0.309645
LR_pcc_sel,0.382866,0.011192,0.379123,0.129272,2.908696,0.010626,2.926368,0.309645
LR_chain_sel,0.382866,0.011192,0.379123,0.129272,2.908696,0.010626,2.926368,0.309645
RanF_ind_sel,0.999592,0.000752,0.789035,0.086480,0.261583,0.004814,1.222281,0.837368
RanF_pcc_sel,0.997786,0.001372,0.793860,0.086916,0.246720,0.003688,1.152814,0.828554
Ranf_chain_sel,0.999592,0.000752,0.788947,0.084291,0.246660,0.004266,1.279247,0.869334
Rufit_pcc_sel,0.957303,0.006573,0.781842,0.070864,0.369302,0.011840,1.035001,0.443024


In [16]:
import pickle
cur_save=open('./' + 'interpolation_select_vwso' + '.pkl', 'rb')
interpolation2 = pickle.load(cur_save)
df2 = interpolation2.summary()
df2 = df2.rename(index={'RF_ind': 'RanF_ind', 'RF_pcc': 'RanF_pcc', 'Rf_chain': 'Ranf_chain'})
df2.index = [each +'_sel_ord' for each in df2.index.tolist()]
df2 # selected predictors

,mean_train_accuracy,std_train_accuracy,mean_test_accuracy,std_test_accuracy,mean_train_log loss,std_train_log loss,mean_test_log loss,std_test_log loss
LR_ind_sel_ord,0.382689,0.011744,0.378509,0.149020,2.908712,0.011742,2.929920,0.345263
LR_pcc_sel_ord,0.382689,0.011744,0.378509,0.149020,2.908712,0.011742,2.929920,0.345263
LR_chain_sel_ord,0.382689,0.011744,0.378509,0.149020,2.908712,0.011742,2.929920,0.345263
RanF_ind_sel_ord,0.999825,0.000533,0.783421,0.070991,0.260965,0.004806,1.201244,0.810822
RanF_pcc_sel_ord,0.999185,0.001099,0.793684,0.085126,0.253450,0.003751,1.101182,0.677113
Ranf_chain_sel_ord,1.000000,0.000000,0.783421,0.074420,0.256048,0.003931,1.243650,0.835603
Rufit_pcc_sel_ord,0.953983,0.006369,0.781930,0.086727,0.381389,0.012505,1.083254,0.463383


In [15]:
import pickle
cur_save=open('./' + 'interpolation_full_swvo' + '.pkl', 'rb')
interpolation3 = pickle.load(cur_save)
df3 = interpolation3.summary()
df3 = df3.rename(index={'RF_ind': 'RanF_ind', 'RF_pcc': 'RanF_pcc', 'Rf_chain': 'Ranf_chain'})
df3.index = [each +'_all' for each in df3.index.tolist()]
df3 # selected predictors

,mean_train_accuracy,std_train_accuracy,mean_test_accuracy,std_test_accuracy,mean_train_log loss,std_train_log loss,mean_test_log loss,std_test_log loss
LR_ind_all,0.383621,0.011012,0.373772,0.073970,2.908707,0.007129,2.928996,0.219702
LR_pcc_all,0.383621,0.011012,0.373772,0.073970,2.908707,0.007129,2.928996,0.219702
LR_chain_all,0.383621,0.011012,0.373772,0.073970,2.908707,0.007129,2.928996,0.219702
RanF_ind_all,0.999592,0.000881,0.780175,0.089267,0.244023,0.004196,1.364488,1.227947
RanF_pcc_all,0.998835,0.001324,0.790351,0.093054,0.234639,0.004447,1.228537,1.142531
Ranf_chain_all,0.999592,0.000881,0.792281,0.086659,0.233098,0.004429,1.404620,1.338222
Rufit_pcc_all,0.955848,0.005486,0.789825,0.091968,0.367529,0.012068,1.100855,0.566046


## Columns:
* _all: __use__ $69$ features and prediction order is __"sphere", "vesicle", "worm", "other"__
* _sel: __use__ $23$ features and prediction order is __"sphere", "vesicle", "worm", "other"__
* _sel_ord: __use__ $23$ features and prediction order is __"vesicle", "worm", "sphere", "other"__

In [17]:
import pandas as pd

df = pd.concat([df1, df2, df3])
df = df.sort_index()
df

,mean_train_accuracy,std_train_accuracy,mean_test_accuracy,std_test_accuracy,mean_train_log loss,std_train_log loss,mean_test_log loss,std_test_log loss
LR_chain_all,0.383621,0.011012,0.373772,0.073970,2.908707,0.007129,2.928996,0.219702
LR_chain_sel,0.382866,0.011192,0.379123,0.129272,2.908696,0.010626,2.926368,0.309645
LR_chain_sel_ord,0.382689,0.011744,0.378509,0.149020,2.908712,0.011742,2.929920,0.345263
LR_ind_all,0.383621,0.011012,0.373772,0.073970,2.908707,0.007129,2.928996,0.219702
LR_ind_sel,0.382866,0.011192,0.379123,0.129272,2.908696,0.010626,2.926368,0.309645
LR_ind_sel_ord,0.382689,0.011744,0.378509,0.149020,2.908712,0.011742,2.929920,0.345263
LR_pcc_all,0.383621,0.011012,0.373772,0.073970,2.908707,0.007129,2.928996,0.219702
LR_pcc_sel,0.382866,0.011192,0.379123,0.129272,2.908696,0.010626,2.926368,0.309645
LR_pcc_sel_ord,0.382689,0.011744,0.378509,0.149020,2.908712,0.011742,2.929920,0.345263
RanF_ind_all,0.999592,0.000881,0.780175,0.089267,0.244023,0.004196,1.364488,1.227947


## Get Baseline

* __Logloss__   
For a single sample with true label $y \in \{0,1\}$ and a probability estimate $p=P(y=1)$, the log loss is:
  
$$L = -(y\log(p) + (1-y)\log(1-p))$$.

* __Error Rate__
For a sample of $n$ observations, the probability of $k$ unique observation is $\{p_1, p_2, \dots, p_k\}$. The error rate is:

$$
E_{rr} = 1 - argmax \{p_i: p_i \in \{p_1, \dots, p_k\}\}
$$


In [18]:
def plot_summary(metric, summ, num_reps=30, baseline=None, names=None, colors = list(mcolors.BASE_COLORS.keys())):
    width = 0.35
    ind = np.arange(len(summ))
    plt.bar(ind-width/2, summ[f'mean_train_{metric}'], width=width, label='train', 
            yerr=summ[f'std_train_{metric}']/num_reps**0.5, capsize=3.0)
    plt.bar(ind+width/2, summ[f'mean_test_{metric}'], width=width, label='test',
            yerr=summ[f'std_test_{metric}']/num_reps**0.5, capsize=3.0)
    if baseline:
        for i in range(len(baseline)):
            plt.axhline(y=baseline[i], color=colors[i], linestyle='-', label=names[i])
    plt.ylabel(metric)
    plt.legend()
    plt.xticks(ind, summ.index)
    
df['mean_train_error'] = 1- df['mean_train_accuracy']
df['std_train_error'] = df['std_train_accuracy']
df['mean_test_error'] = 1- df['mean_test_accuracy']
df['std_test_error'] = df['std_test_accuracy']

In [19]:
# base line
uniq, cnt = np.unique(data.y.values, axis=0, return_counts=True) 
prob = cnt/sum(cnt)
info_logloss = sum(-1*prob * np.log2(prob))
print('Informed logloss: ', info_logloss)

uniprob = np.array([1/16 for _ in range(16)])
uninfo_logloss =  sum(-1*uniprob * np.log2(uniprob))
print('Uninformed logloss: ', uninfo_logloss)

Informed logloss:  2.5225679165070907
Uninformed logloss:  4.0


In [20]:
from matplotlib.pyplot import figure
figure(figsize=(100, 40), dpi=200)
plt.rcParams["font.size"] = "25"
plot_summary('log loss', summ=df, baseline=[info_logloss, uninfo_logloss], names=['Informed', 'Uninformed'])

In [21]:
# Error rate base line
uniq, cnts = np.unique(data.y.values, axis=0, return_counts=True)
prob = cnts/sum(cnts)
informed_error = 1- max(prob)
print('Informed Error: ', informed_error)

uninformed_error = 1-1/16
print('Uninformed Error: ', uninformed_error)

Informed Error:  0.5912162162162162
Uninformed Error:  0.9375


In [22]:
figure(figsize=(100, 40), dpi=200)
plt.rcParams["font.size"] = "25"
plot_summary('error', df, baseline=[informed_error, uninformed_error], names=['Informed', 'Uninformed']) # below plot is the error rate (1- accuracy)